### RLHF to check ACD works
This notebook uses ACD (Adversarial Contrastive Distillation) a HELLASWAG inspired data generation to fit RLHF. It also uses a portion of the dataset to measure model performance improvements.

In [2]:
from pathlib import Path
import json
import evaluate

from datasets import load_dataset

from transformers import (pipeline, AutoTokenizer, AutoModelForSequenceClassification,
                          AutoModelForSeq2SeqLM, GenerationConfig)
from peft import PeftModel, PeftConfig, LoraConfig, TaskType
import torch, random, numpy as np, pandas as pd

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

from shared_models import HellaSwagEntry

from tqdm import tqdm
tqdm.pandas()

### Load Data

In [3]:
DATA_PATH = "../data/hellaswag_format/personal_chat_sessions_train_hellaswag.jsonl"

In [ ]:
def load_jsonl_pydantic(path: Path):
    """Yield HellaSwagEntry objects parsed with Pydantic."""
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            yield HellaSwagEntry.model_validate_json(line)

records = list(load_jsonl_pydantic(Path(DATA_PATH)))
print(f"Loaded {len(records):,} examples")